In [131]:
#export
import json
import sys
import os
import os.path
from pathlib import Path
from subprocess import call
import subprocess
import sys # system libraries, like arguments (argv)
import re # regular expressions
import pandas as pd
import glob
import os.path
from pathlib import Path
# from nb_FilterLastDB import *


from timeit import default_timer as timer
import time



In [132]:
#export
def makeFolders(folder_list):
    for directory in folder_list:
        if not directory: # Make sure not an empty string ""
            continue
        else:
            if not os.path.exists(directory):
                os.makedirs(directory)

### Filter class (Filter contigs of interest)

In [133]:
#export
class JSON():
    def __init__(self, f, explicit, min_length = 100):
        self.f = f
        self.min_length = min_length
        self.json_empty = False
        self.num_contigs = 0
        self.explicit = explicit
        with open(f) as fo:
            cd = json.load(fo) # cd is contig dictionary
            k = list(cd.keys())
            self.pe(f'Search starting with {len(k)} contigs')
            if len(k) > 0:
                self.filename = cd[k[0]]["filename"]
                self.cd = cd
            else:
                self.json_empty = True
                self.cd = {}
                self.pe(f'*-**-**-**-**-**-**-**-**-**-* WARNING file Empty!!!')
            
    def pe(self, s):
        if self.explicit:
            print(s)
            
            
    def updateCD(self):
        self.k = list(self.cd.keys())
        self.num_contigs = len(self.k)
        if self.num_contigs < 1:
            self.json_empty = True
    
    def minLength(self):
        if self.json_empty == False:
            cd = self.cd.copy()
            for c in list(self.cd):
                length = int(self.cd[c]["length"])
                if length<self.min_length:
                    self.cd.pop(c) 
            self.updateCD()
            self.pe(f'{len(self.k)} contigs after filtering for min length {min_length} (Default 100)')
            if J.json_empty: self.pe(f'WARNING!!! Found no contigs after filtering for min length')

        
               
    def searchRank(self, rank, HAS=True):
        if self.json_empty == False:
            cd_rank = {}
            cd_norank = {}
            for c in list(self.cd):
                contig = self.cd[c]
                try:
                    rd = self.cd[c]["jgi_json"][rank]
                    cd_rank[c] = contig
                except:
                    cd_norank[c] = contig
            if HAS:
                self.cd = cd_rank
            else:
                self.cd = cd_norank
            self.updateCD()
        else:
            self.pe("No input into rank search")
            self.updateCD()
            
    
    def searchRankName(self, rank, name, HAS=True):
        if self.json_empty == False:
            self.searchRank(rank=rank, NOT=False)
            if self.json_empty == False:
                for c in list(self.cd):
                    r_d = self.cd[c]["jgi_json"][rank]
                    if HAS:
                        if r_d["name"]!=name:
                            self.cd.pop(c)   
                    else:
                        if r_d["name"]==name:
                            self.cd.pop(c)                      
                self.updateCD()
            else:
                self.pe(f"Nothing in rank {rank}")
                self.updateCD()
        else: 
            self.pe("No input into searchRankName")
            self.updateCD()
            
            
    
    def searchJ(self,rank="NA", name="NA", HAS=True, FILTER=True):
        if FILTER:
            if self.json_empty: self.pe(f'json empty')
    #         print(f'Removing contigs less than {self.min_length}')
            if rank != "NA" and name != "NA":  
                self.searchRankName(rank=rank, name=name, HAS=HAS)
                self.pe(f"{self.num_contigs}:contigs after search for...\ncontains({HAS}) rank:{rank}\n  name:{name}")        
            if rank != "NA" and name == "NA":  
                self.searchRank(rank=rank, HAS=HAS)  
                self.pe(f"{self.num_contigs}:contigs after search for...\ncontains({HAS})  rank:{rank}")
            print(f'{len(self.k)} contigs after filtering')
        self.updateCD()
    


In [134]:
rank = "kingdom"
label = "Metazoa"
min_length = 100

p = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/nextflow/RNAseq-Biome-Nextflow/bin/test/jgi_test/"
f1=p+"test_jgi.json"    

# J = makeJ(f, min_length, NOT=False)
# J = makeJ(f, min_length, NOT=False, rank="phylum")
# self.cd = self.searchRankName(self.cd,'kingdom','Metazoa')
# J = makeJ(f, min_length, NOT=True, rank="kingdom", name = 'Metazoa')


J = JSON(f=f1, min_length=min_length, explicit=True)
J.searchJ(HAS=False, rank="phylum")
# J = searchJ(J, min_length, NOT=True, rank="species", name = 'eukaryotic synthetic construct')
# J = searchJ(J, min_length, NOT=True, rank="phylum", name = 'Platyhelminthes')
# J = searchJ(J, min_length, NOT=True, rank="phylum", name  = 'Platyhelminthes')
# J = searchJ(J, min_length, NOT=False, rank="phylum")
# J = searchJ(J, min_length, NOT=False, rank="species", name = 'eukaryotic synthetic construct')
# J = searchJ(J, min_length, NOT=False, rank="species", name = 'eukaryotic synthetic construct')


# Not will remove everything not in the Rank. 
# If you include name it will return the rank and remove everything that doesnt have that name


J.num_contigs
J.cd

Search starting with 3 contigs
1:contigs after search for...
contains(False)  rank:phylum
1 contigs after filtering


{'2': {'contig': 'NODE_26_length_223_cov_282.690476_g25_i0',
  'blast_pident': 'gi|1548994288|gb|CP034492.1|',
  'blast_sseqid': '100.000',
  'blast_evalue': '4.98e-83',
  'taxid': '111789',
  'blast_stitle': 'Eukaryotic synthetic construct chromosome 14',
  'filename': 'test',
  'fasta_with_returnsAdded': 'GCATACGCGATCAGTTCCGGTGACTGGAGTTCGACGTGTGCTCTTCCGATCCGGTGGCGCRTN666GTGCCTGTAGTCCCAGCTACTCGGGAGGCTGAGGCTGGAGGATCGCTTGAGTCCAGGAGTRTN666TCTGGGCTGTAGTGCGCTATGCCGATCGGGTGTCCGCACTAAGTTCGGCATCAATATGGTRTN666GACCTCCCGGGAGCGGGGGACCACCAGGTTGCCTAAGGAGGGGRTN666',
  'fasta': 'GCATACGCGATCAGTTCCGGTGACTGGAGTTCGACGTGTGCTCTTCCGATCCGGTGGCGCGTGCCTGTAGTCCCAGCTACTCGGGAGGCTGAGGCTGGAGGATCGCTTGAGTCCAGGAGTTCTGGGCTGTAGTGCGCTATGCCGATCGGGTGTCCGCACTAAGTTCGGCATCAATATGGTGACCTCCCGGGAGCGGGGGACCACCAGGTTGCCTAAGGAGGGG',
  'length': 223,
  'jgi_json': {'name': 'eukaryotic synthetic construct',
   'tax_id': 111789,
   'level': 'species',
   'species': {'name': 'eukaryotic synthetic construct', 'tax_id': 111789},
   'no ra

## Concat Jsons after query and make gtf

In [135]:
#export
class Quantify():
    
    def __init__(self, NF_out, title, searchlist, 
                 NF_path, col_data, cpus, 
                 singleEnd,  explicit, out_path,
                 control, treatment, binContigs,  min_length,
                 binRank="blank"):
        self.binContigs = binContigs
        self.binRank = binRank
        self.control = control
        self.treatment = treatment
        self.explicit = explicit
        self.cpus = cpus
        self.singleEnd = singleEnd
        self.NF_out = NF_out
        self.NF_path = NF_path
        self.wkdir = f'{self.NF_out}/unmapped/final'
        self.title = title
        self.Jlist_d = {}
        self.df_norm = pd.DataFrame()
        self.out_path = out_path
        self.searchlist = searchlist
        self.col_data = col_data
        self.min_length = min_length
        self.df_col = pd.read_csv(self.col_data,sep="\t",names = ["sample","condition"])
        self.condition_list = self.df_col["condition"].unique()
#         self.levels = ["single","group","all",
#                        "dark_single_FG","dark_group_FA","dark_all_FG"]
        self.levels = ["all","dark_all_FG"]

    def pe(self, s):
        if self.explicit:
            print(s)        
    
        
        

    def toJSON(self):
        import ast
        
        for f in glob.glob(f'{self.wkdir}jgi_df/*/*'):
#         for f in glob.glob(f'{self.wkdir}+jgi_df/*/*')
            l = len("_df.txt")
            fout = f[:-l]
        
            df_J = df[["taxid","jgi_json"]]
            del df["jgi_json"]
            df.to_json(f"{fout}_temp.json",orient='index')
            with open(f"{fout}_temp.json") as fo:
                contig_d = json.load(fo)
                for c in contig_d.keys():
                    d = contig_d[c] 
                    taxid = d["taxid"]  
                    df_temp = df_J[df_J["taxid"]==taxid]
                    j = df_temp["jgi_json"].iloc[0]
                    jdic = ast.literal_eval(j)
                    d["jgi_json"] = jdic
                    d["filename"] = name

                with open(f"{fout}.json", "w") as write_file:
                    json.dump(contig_d, write_file)
            removeFile(f"{fout}_temp.json")
            
        
        
    
    def searchContigs(self):
        for level in self.levels:
            Jlist = []
            if self.explicit:
                print(f'Working on {level} --- ')
            if level == "single" or "group" or "all":
                jsons = glob.glob(f'{self.wkdir}/jgi_df/{level}/*.json')
            
            #Dark genome
            folder = "darkbiome/lastdb/FINAL_Tophits"
            
#             
#             /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/Z_RNAseq-Biome-master/NF_OUT/unmapped/final/darkbiome/lastdb/FINAL_Tophits/all_group/all/all_darkbiome_df.txt
            if level == "dark_single_FG":
                jsons = glob.glob(f'{self.wkdir}/{folder}/group_single/single/*.json')               
            if level == "dark_group_FA":
                jsons = glob.glob(f'{self.wkdir}/{folder}/all_group/group/*.json')                   
            if level == "dark_all_FG":
                jsons = glob.glob(f'{self.wkdir}/{folder}/all_group/all/*.json')    
            
            
#             print(f'{self.wkdir}/{folder}/all_group/all/')
#             print(jsons)
                
            for f in jsons:
                print("---------------")
                print(f' Working on file {f}')
                J = JSON(f=f, explicit=self.explicit)
#                 J.minLength()

                if J.json_empty == False:
                    Jlist.append(J)  
            
            if len(Jlist) < 1:
                print("*_*_*_*_*_*_*_ WARNING *_*_*_*_*_*_*_")
                print(f"No files in {level} had any contigs")
            else:
                self.Jlist_d[level] = Jlist
                
                
                
                
                
    def makeGTF_and_df(self):
        makeFolders([f'{self.out_path}/{self.title}'])
        for level in self.levels:
            out = f'{self.out_path}/{self.title}/{level}'
            makeFolders([out])
            f_gtf = f'{out}/{level}_{self.title}.gtf'
            f_df = f'{out}/{level}_{self.title}.tsv'
            
            open(f_gtf,"w")
            df_final = pd.DataFrame()
            with open(f_gtf,"a") as outfile:
                for J in self.Jlist_d[level]:
                    for c in J.cd:
                        contig_d = J.cd[c]
                        contig_name = contig_d["contig"]
                        length = contig_d["length"]
                        if self.binRank != "NA":
                            binrank = contig_d['jgi_json'][self.binRank]["name"]
                        else:
                            binrank = contig_name
                        line = f'{contig_name}\tblank\texon\t1\t{length}\t.\t+\t.\t{self.binRank} "{binrank}"\n'
                        outfile.write(line)
                    df = pd.DataFrame.from_dict(J.cd).T
                    df = df[["filename","contig","length","taxid","jgi_tab","fasta"]]
                    df_final = pd.concat([df_final,df])
            df_final.to_csv(f_df,sep="\t",index=None)
            if len(df_final) > 0:
                self.runR(out = out, level=level, gtf = f_gtf)
            else:
                with open(f'{self.out_path}/{self.title}/Empty_files.txt',"w") as outfile:
                    outfile.write(os.path.basename(f_gtf)+"\n")
            
            
    def get_star_output(self):
        """This function will return a df from star of all reads"""
        starpath = f'{self.NF_out}/qc/star_mapstats'
        filelist = glob.glob(starpath+"/*Log.final.out")
        df = pd.DataFrame(columns=["sample","input","unique","multi"])
        x = 0
        
        
        for f1 in filelist:
            with open(f1,"r") as infile:
                input_reads_list = []
                unique_reads_list = []
                multi_reads_list = []
                for line in infile:
                    if "Number of input reads" in line:
                        scl = [m.start() for m in re.finditer(r"\t",line)]  
                        input_reads = line[ scl[0]:].strip("\t")
                        input_reads_list.append(int(input_reads))
                    if "Uniquely mapped reads number" in line:
                        scl = [m.start() for m in re.finditer(r"\t",line)]  
                        unique_reads = line[ scl[0]:].strip("\t")
                        unique_reads_list.append(int(unique_reads))
                    if "Number of reads mapped to multiple loci" in line:
                        scl = [m.start() for m in re.finditer(r"\t",line)]  
                        multi_reads = line[ scl[0]:].strip("\t")
                        multi_reads_list.append(int(multi_reads))
                sample = os.path.basename(f1).replace("Log.final.out","")
                df.loc[x] = pd.Series({"sample":sample, "unique": unique_reads_list[0], "input":input_reads_list[0],"multi":multi_reads_list[0]})
            x = x + 1

        df = get_star_output()
        df["total_mapped"] = (df["unique"] + df["multi"]).astype(int)
        df["unmapped"] = df["input"] - df["total_mapped"]

        df["total_mapped_avg"] = df["total_mapped"].mean()
        df["total_mapped/total_mapped_avg"] = df["total_mapped"] / df["total_mapped_avg"]            
        df.to_csv(starpath+"/starmapstats.txt",sep="\t",index=None)
        self.df_norm = df
        
            
            
    def runR(self, out, level, gtf):
        if self.singleEnd:
            PE = "FALSE"
        else:
            PE = "TRUE"
            
        if self.binContigs:
            MF = "TRUE"
        else:
            MF = "FALSE"                  
    
        #Make col data file
        if level == "single" or level =="dark_single_FG":
            bamfront = f'{self.wkdir}/bam/single/'
            bamend = "_unmapped_single.bam"

        if level == "group" or level =="dark_group_FA":
            bamfront = f'{self.wkdir}/bam/group/'
            bamend = "_unmapped_group.bam"
            
        if level == "all" or level =="dark_all_FG":
            bamfront = f'{self.wkdir}/bam/all/'
            
#             Biome-master/NF_OUT/unmapped/final/bam/all
            bamend = "_unmapped_all.bam"            
            
        df = self.df_col.copy()
        
        # Run on all samples
#         df = df.loc[df['condition'].isin([self.treatment,self.control])]
#         print(df)
        df["sample2"] = bamfront + df["sample"] + bamend
        #Make bamlist for dseq
        bamlist = df["sample2"].tolist()
        bamlist = str(bamlist).strip("[").strip("]")
    
        # fix col data to match rsubread out
#         X513.S37.L002.unmapped.all.bam
        df["sample"] = "X"+df["sample"].str.replace("_",".")+bamend.replace("_",".")
        del df["sample2"]
        df.to_csv(f'{out}/col_data.txt',sep="\t",index=None, header=None)
    
    
            
        #Make template subread and r script
        template = f'{self.NF_path}/bin/R_subread_DEseq2_TEMPLATE.R'
        rscript = f'{out}/{level}.R'
        with open(template, "r") as infile, open(rscript, "w") as outfile:
            for line in infile:
                line = line.replace("bamlist", bamlist)
                line = line.replace("outputprefix", out)
                line = line.replace("COL_DATA", f'{out}/col_data.txt')
                line = line.replace("PAIRED", PE)
                line = line.replace("BLANK", self.binRank)
                line = line.replace("METAFEATURE", MF)
                line = line.replace("CONTROL", self.control)
                line = line.replace("TREATMENT", self.treatment)
                line = line.replace("temp_annotation", str(gtf))
                line = line.replace("cpus", str(self.cpus))
                outfile.write(line)
                #R CMD BATCH /D_nonsig.R ${COMPARISON_PATHsig.R.out
        cmd = f'R CMD BATCH {rscript} {rscript}.R.out'
        subprocess.run(cmd, shell=True)
        print("done with r")

 

In [137]:
NF = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master"
NF_out = f"{NF}/NF_OUT" 
NF_path = NF
out_path = NF
col_data = f"{NF}/sample_table.txt"
singleEnd = False
control = "one_POLYA"
treatment = "TW4_POLYA"
# searchlist = [(True,"phylum","NA")]
searchlist = None
binRank = "NA"
binContigs = False
min_length = 100
# searchlist = [(True,"phylum","NA"), (False,"phylum","Bacteria_name")]





# Q = Quantify(NF_out=NF_out, 
#              NF_path=NF_path,
#              out_path = out_path, 
#              col_data = col_data, 
#              cpus=4, 
#              singleEnd=False, 
#              title="HasPhylum", 
#              searchlist=searchlist,
#              control = control, 
#              treatment = treatment, 
#              explicit=False,
#              binContigs = binContigs,
#              binRank = binRank,
#              min_length = min_length
#             )
# Q.searchContigs()
# Q.makeGTF_and_df()




### RSubread template script


### Run RSubread and DESeq2 



## Graphing class

In [138]:
#export
import argparse
def parse_arguments():
        parser = argparse.ArgumentParser(description='filter quantify and graph')
        parser.add_argument('--col_data', action= 'store', metavar='col_data') 
#         parser.add_argument('--f', action= 'store', metavar='f') 
        parser.add_argument('--Nextflow_Out', action= 'store', metavar='--Nextflow_Out') 
        parser.add_argument('--singleEnd', action= 'store', metavar='--singleEnd') 
        parser.add_argument('--Nextflow_path', action= 'store', metavar='--Nextflow_path')
        parser.add_argument('--Control', action= 'store', metavar='--Control')
        parser.add_argument('--Treatment', action= 'store', metavar='--Treatment')
        parser.add_argument('--cpus', action= 'store', metavar='--cpus')
        
        args = parser.parse_args()
        return args


In [141]:
#export
if __name__=="__main__":
    args = parse_arguments()
#     f = args.f
    NF_out = args.Nextflow_Out
    NF_path = args.Nextflow_path
    singleEnd = args.singleEnd
    Control = args.Control
    Treatment = args.Treatment
    cpus = args.cpus

    out_path = NF_path
    
    
    
    col_data = f"{NF_path}/sample_table.txt"
    levels = ["all","dark_all_FG"]
    singleEnd = False
 


    searchlist = None
    binRank = "NA"
    binContigs = False
    min_length = 100
    title = "ALL"
    # searchlist = [(True,"phylum","NA"), (False,"phylum","Bacteria_name")]
    Q = Quantify(NF_out=NF_out, 
                 NF_path=NF_path,
                 out_path = out_path, 
                 col_data = col_data, 
                 cpus=cpus, 
                 singleEnd=False, 
                 title=title, 
                 searchlist=searchlist,
                 control = Control, 
                 treatment = Treatment, 
                 explicit=False,
                 binContigs = binContigs,
                 binRank = binRank,
                 min_length = min_length
                )
    Q.searchContigs()
    Q.makeGTF_and_df()






    
    

usage: ipykernel_launcher.py [-h] [--col_data col_data]
                             [--Nextflow_Out --Nextflow_Out]
                             [--singleEnd --singleEnd]
                             [--Nextflow_path --Nextflow_path]
                             [--Control --Control] [--Treatment --Treatment]
                             [--cpus --cpus]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/m/Library/Jupyter/runtime/kernel-9afbd306-675a-4ac3-b94f-7775feb0d421.json


SystemExit: 2

In [142]:
import fire
!python notebook2script.py FilterQuantSubreadDESeq.ipynb

Converted FilterQuantSubreadDESeq.ipynb to nb_FilterQuantSubreadDESeq.py
